# Spark Initialization

In [1]:
# Import findspark to read SPARK_HOME and HADOOP_HOME
import findspark
findspark.init()

In [2]:
# Import required library
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession.builder.appName("Global Commodity Trade Statistics").getOrCreate()

# Print Spark object ID
print(spark)

# Loading Data using Spark

In [3]:
# Dataset could be downloaded at https://www.kaggle.com/unitednations/global-commodity-trade-statistics
df = spark.read.csv("D:/kuliah/bigdata/commodity_trade_statistics_data.csv", header=True, inferSchema=True)

In [4]:
df.show()

+---------------+----+---------+--------------------+------+---------+---------+---------------+--------+---------------+
|country_or_area|year|comm_code|           commodity|  flow|trade_usd|weight_kg|  quantity_name|quantity|       category|
+---------------+----+---------+--------------------+------+---------+---------+---------------+--------+---------------+
|    Afghanistan|2016|   010410|         Sheep, live|Export|     6088|     2339|Number of items|      51|01_live_animals|
|    Afghanistan|2016|   010420|         Goats, live|Export|     3958|      984|Number of items|      53|01_live_animals|
|    Afghanistan|2008|   010210|Bovine animals, l...|Import|  1026804|      272|Number of items|    3769|01_live_animals|
|        Albania|2016|   010290|Bovine animals, l...|Import|  2414533|  1114023|Number of items|    6853|01_live_animals|
|        Albania|2016|   010392|Swine, live excep...|Import| 14265937|  9484953|Number of items|   96040|01_live_animals|
|        Albania|2016|  

In [5]:
df.count()

8226597

In [6]:
df.schema

StructType(List(StructField(country_or_area,StringType,true),StructField(year,IntegerType,true),StructField(comm_code,StringType,true),StructField(commodity,StringType,true),StructField(flow,StringType,true),StructField(trade_usd,LongType,true),StructField(weight_kg,LongType,true),StructField(quantity_name,StringType,true),StructField(quantity,DecimalType(20,0),true),StructField(category,StringType,true)))

In [7]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("commodity_trade")

In [8]:
result = spark.sql("SELECT DISTINCT commodity FROM commodity_trade")

In [9]:
result.show()

+--------------------+
|           commodity|
+--------------------+
|Ash or residues c...|
|Coal briquettes, ...|
|Amino-naphthols/p...|
|Polyesters nes, u...|
|Filament tow of a...|
|Twine, cordage, r...|
|Woven pile fabric...|
|Mens, boys underp...|
|Mens, boys dressi...|
|Used or new rags ...|
|Ceramic houseware...|
|Blades for kitche...|
|Machinery to reel...|
|Coal or rock cutt...|
|Primary cells, pr...|
|Brakes except air...|
|Thermo-copying ap...|
| Cheese, blue-veined|
|Hair, human, unwo...|
|Sunflower or saff...|
+--------------------+
only showing top 20 rows



# Data Mining Process

In [10]:
# 1. Find max commodity trade in USD by country
query1 = spark.sql("SELECT a.country_or_area, a.year, a.commodity, a.flow, a.trade_usd \
FROM commodity_trade a \
INNER JOIN ( \
    SELECT MAX(trade_usd) AS maxtrade, country_or_area \
    FROM commodity_trade \
    WHERE comm_code != 'TOTAL' \
    GROUP BY country_or_area) b \
ON a.trade_usd = b.maxtrade \
AND a.country_or_area = b.country_or_area \
ORDER BY a.country_or_area")

In [11]:
query1.show()

+-------------------+----+--------------------+------+-----------+
|    country_or_area|year|           commodity|  flow|  trade_usd|
+-------------------+----+--------------------+------+-----------+
|        Afghanistan|2013|Commodities not s...|Import| 5857516532|
|            Albania|2014|Commodities not s...|Import| 2250573559|
|            Algeria|2008|Petroleum oils, o...|Export|41649283779|
|            Andorra|2012|Oils petroleum, b...|Import|  154268123|
|             Angola|2012|Petroleum oils, o...|Export|68863266749|
|           Anguilla|2008|Commodities not s...|Import|   73431281|
|Antigua and Barbuda|2009|Commodities not s...|Import|  267436094|
|          Argentina|2014|Soya-bean oil-cak...|Export|11837635727|
|            Armenia|2013|Natural gas in ga...|Import|  571766320|
|              Aruba|2007|Commodities not s...|Import| 1113876882|
|              Aruba|2007|     ALL COMMODITIES|Import| 1113876882|
|          Australia|2013|Iron ore, concent...|Export|66917183

In [12]:
# 2. Sum total trade by USD for each year
query2 = spark.sql("SELECT year, SUM(trade_usd) \
FROM commodity_trade \
GROUP BY year \
ORDER BY year")

In [13]:
query2.show()

+----+--------------+
|year|sum(trade_usd)|
+----+--------------+
|null|          null|
|1988| 1919212691750|
|1989| 3121488716188|
|1990| 3661899612274|
|1991| 4997632816808|
|1992| 6356089390115|
|1993| 7501579992911|
|1994| 9992199334037|
|1995|12536445490812|
|1996|13383105467754|
|1997|14163002577080|
|1998|13988907513103|
|1999|14496904257308|
|2000|19324976747277|
|2001|18758192334353|
|2002|19762219985016|
|2003|23115762568721|
|2004|28122945047934|
|2005|31688587865168|
|2006|37105513485784|
+----+--------------+
only showing top 20 rows



In [14]:
# 3. Select most traded commodity in USD for each year
query3 = spark.sql("SELECT a.year, a.commodity, a.max_trade_usd \
FROM ( \
    SELECT year, commodity, SUM(trade_usd) AS max_trade_usd \
    FROM commodity_trade \
    GROUP BY commodity, year) a \
INNER JOIN ( \
    SELECT year, MAX(sum_trade_usd) AS max_trade_usd \
    FROM ( \
        SELECT year, commodity, SUM(trade_usd) AS sum_trade_usd \
        FROM commodity_trade \
        WHERE comm_code != 'TOTAL' \
        GROUP BY year, commodity) \
    GROUP BY year) b \
ON a.year = b.year \
AND a.max_trade_usd = b.max_trade_usd \
ORDER BY a.year")

In [15]:
query3.show()

+----+--------------------+-------------+
|year|           commodity|max_trade_usd|
+----+--------------------+-------------+
|1988|Petroleum oils, o...|  31491572782|
|1989|Petroleum oils, o...|  56593730006|
|1990|Petroleum oils, o...|  79537679505|
|1991|Commodities not s...|  90353423662|
|1992|Commodities not s...| 120430688413|
|1993|Commodities not s...| 151068532283|
|1994|Commodities not s...| 166493377181|
|1995|Commodities not s...| 225939047916|
|1996|Commodities not s...| 255739729899|
|1997|Commodities not s...| 268421077555|
|1998|Commodities not s...| 238647823138|
|1999|Commodities not s...| 255391416670|
|2000|Commodities not s...| 588660760125|
|2001|Commodities not s...| 534924276270|
|2002|Commodities not s...| 635556526098|
|2003|Commodities not s...| 748332948062|
|2004|Commodities not s...| 937104726181|
|2005|Petroleum oils, o...| 803185443958|
|2006|Commodities not s...|1178855125851|
|2007|Petroleum oils, o...|1154022544567|
+----+--------------------+-------

In [16]:
# 4. Select exported commodities by country
query4 = spark.sql("SELECT country_or_area, commodity \
FROM commodity_trade \
WHERE flow = 'Export' \
GROUP BY commodity, country_or_area \
ORDER BY country_or_area, commodity")

In [17]:
query4.show()

+---------------+--------------------+
|country_or_area|           commodity|
+---------------+--------------------+
|    Afghanistan|     ALL COMMODITIES|
|    Afghanistan|Almonds in shell ...|
|    Afghanistan|Almonds,fresh or ...|
|    Afghanistan|Antiques older th...|
|    Afghanistan|     Apricots, dried|
|    Afghanistan|     Apricots, fresh|
|    Afghanistan|Articles for pock...|
|    Afghanistan|              Barley|
|    Afghanistan|Brazil nuts, fres...|
|    Afghanistan|Carpets of wool o...|
|    Afghanistan|Cashew nuts, fres...|
|    Afghanistan|Cauliflowers and ...|
|    Afghanistan|Chemical industry...|
|    Afghanistan|Citrus juice nes ...|
|    Afghanistan|Coal except anthr...|
|    Afghanistan|Commodities not s...|
|    Afghanistan|     Coriander seeds|
|    Afghanistan|         Cumin seeds|
|    Afghanistan|Emery & natural a...|
|    Afghanistan|Figs, fresh or dried|
+---------------+--------------------+
only showing top 20 rows

